In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers.trainer_utils import EvaluationStrategy
import os
import torch

In [2]:
df = pd.read_csv("/content/prufrock_metaphor_conversation_final.csv")

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision_literal": precision[0],
        "recall_literal": recall[0],
        "f1_literal": f1[0],
        "precision_metaphor": precision[1],
        "recall_metaphor": recall[1],
        "f1_metaphor": f1[1],
    }


In [7]:
os.environ["WANDB_DISABLED"] = "true"  # disable W&B logging

training_args = TrainingArguments(
    output_dir="./models/prufrock_roberta",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_metaphor"
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2622461419.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision Literal,Recall Literal,F1 Literal,Precision Metaphor,Recall Metaphor,F1 Metaphor
1,No log,0.713820,0.307692,0.307692,1.000000,0.470588,0.000000,0.000000,0.000000
2,0.701600,0.662993,0.769231,1.000000,0.250000,0.400000,0.750000,1.000000,0.857143
3,0.670900,0.617127,0.769231,1.000000,0.250000,0.400000,0.750000,1.000000,0.857143
4,0.670900,0.548130,0.769231,1.000000,0.250000,0.400000,0.750000,1.000000,0.857143
5,0.558600,0.421085,0.846154,1.000000,0.500000,0.666667,0.818182,1.000000,0.900000
6,0.420300,0.346736,0.846154,1.000000,0.500000,0.666667,0.818182,1.000000,0.900000


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/py

Evaluation results: {'eval_loss': 0.42108476161956787, 'eval_accuracy': 0.8461538461538461, 'eval_precision_literal': 1.0, 'eval_recall_literal': 0.5, 'eval_f1_literal': 0.6666666666666666, 'eval_precision_metaphor': 0.8181818181818182, 'eval_recall_metaphor': 1.0, 'eval_f1_metaphor': 0.9, 'eval_runtime': 7.0032, 'eval_samples_per_second': 1.856, 'eval_steps_per_second': 0.286, 'epoch': 6.0}


In [12]:
trainer.save_model("./models/prufrock_roberta")


In [15]:
model.eval()

def predict_sentences(sentences):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
    for sent, pred in zip(sentences, predictions.cpu().numpy()):
        print(f"Sentence: {sent}\n→ Prediction: {'Metaphor' if pred == 1 else 'Literal'}\n")

In [17]:
test_sentences = [
    "I have measured out my life with coffee spoons.",
    "He submitted the assignment on time.",
    "She's carrying emotional baggage from college.",
    "life is a river."
]
predict_sentences(test_sentences)

Sentence: I have measured out my life with coffee spoons.
→ Prediction: Metaphor

Sentence: He submitted the assignment on time.
→ Prediction: Literal

Sentence: She's carrying emotional baggage from college.
→ Prediction: Metaphor

Sentence: life is a river.
→ Prediction: Metaphor

